# Daily Shelter Occupancy in the City of Toronto

## Import necessary modules

In [2]:
import pandas as pd
import IPython
import plotly.graph_objects as go
import plotly.express as px

## Load data

In [3]:
# Loading data from previous years
df_17 = pd.read_csv('data/daily-shelter-occupancy-2017-csv.csv')
df_18 = pd.read_csv('data/daily-shelter-occupancy-2018-csv.csv')

In [4]:
# Reading latest data from City of Torotno (updated daily)
try:
    df_current = pd.read_json('https://secure.toronto.ca/c3api_data/v2/DataAccess.svc/ssha/extractssha?$format=application/json;odata.metadata=none&unwrap=true&$top=100000&$select=OCCUPANCY_DATE,ORGANIZATION_NAME,SHELTER_NAME,SHELTER_ADDRESS,SHELTER_CITY,SHELTER_PROVINCE,SHELTER_POSTAL_CODE,FACILITY_NAME,PROGRAM_NAME,SECTOR,OCCUPANCY,CAPACITY&$orderby=OCCUPANCY_DATE,ORGANIZATION_NAME,SHELTER_NAME,FACILITY_NAME,PROGRAM_NAME')
except:
    print('Something wrong with the data source')

## Data Processing

In [5]:
# concatenate 3 data frames 
df = pd.concat([df_17, df_18, df_current], sort=False)

In [6]:
# Dropping unnecessary columns
df = df.drop(['id', '_id'], axis=1)

## Visualizations

### Shleter locations on Google Maps

In [6]:
url = 'https://www.google.com/maps/d/embed?mid=1vjoRnGFoyIIwg8O95s-FITXUOQzTunce'
IPython.display.IFrame(url, width=640, height=480)

### Number of Facilities by Type

In [7]:
# Get unique combination of sector and facility name
sector_facilities = df_17.groupby(['SECTOR', 'FACILITY_NAME']).size().reset_index()

# Get number of facilities by sector
facility_count_by_sector = sector_facilities.drop(0, axis=1).groupby(['SECTOR']).count().reset_index()
facility_count_by_sector

,SECTOR,FACILITY_NAME
0,Co-ed,15
1,Families,18
2,Men,20
3,Women,17
4,Youth,15


In [11]:
fig = px.bar(
    facility_count_by_sector, 
    x='SECTOR', 
    y='FACILITY_NAME',
    color='FACILITY_NAME',
    labels={'FACILITY_NAME': 'Facility Count'}
)

fig.update_layout(title_text='Facility Count by Sector')

fig.show()

In [10]:
# Share of facility types with pie chart
fig = go.Figure(
    data=[go.Pie(
        labels=facility_count_by_sector.SECTOR, 
        values=facility_count_by_sector.FACILITY_NAME
    )])

fig.update_layout(title_text='Facility Share by Sector')

fig.show()